In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# Import Dataset, Processing

In [5]:
DF = pd.read_csv('gurobiTime.csv',index_col=0)
DF = DF.drop(['player.1'],axis=1)


positionDF = []
num_positions = len(DF['position'].unique())


for i in DF['position'].unique():
    positionDF.append( (DF[DF['position'] == i]) )

    

Make Gurobi datastructures to help represent the problem

We create dictionaries for every single variable for each player, to allow for efficient access and ease of use.  The keys are Players, the tupleList objects are returnPlayer, playerVariance, playerCost , and position


QB, RB, RB, WR, WR, WR, TE, D, K 



In [59]:
bigDict = {}
for df_iter in range(len(positionDF)):
    
    currPosition = positionDF[df_iter]  
    for player in currPosition.index:
        returnPlayer = currPosition.loc[player]['mean']
        playerVariance = currPosition.loc[player]['variance']
        playerCost = currPosition.loc[player]['cost']
        position = currPosition.loc[player]['position']
        bigDict[player] = [returnPlayer, playerVariance, playerCost , position]
         
player, returnPlayer, playerVariance, playerCost , position = gp.multidict(bigDict)


positionPlayerConstraintVars = gp.tuplelist( [(position.select(playr)[0],playr) for playr in player])

playerDecisionVars = gp.tuplelist( [ (choice, returnPlayer.select(choice)[0], playerVariance.select(choice)[0])  for choice in player ] )

positionConstraints = gp.tupledict( { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1})#'K':0} )




In [60]:
positionPlayerConstraintVars



# Constraints: assign exactly shiftRequirements[s] workers to each shift s

# reqCts = {}
# for s in shifts:
#   reqCts[s] = m.addConstr(
#        gp.quicksum(x[w,s] for w,s in availability.select('*', s)) ==
#        shiftRequirements[s], s)
positionConstraints

{'QB': 1, 'RB': 3, 'WR': 3, 'TE': 1, 'DEF': 1}

In [70]:
try:
    m = gp.Model('QMIP')
    #add binary decision variables on whether 
    
    
    #add decision variables
    players = m.addVars( playerDecisionVars, vtype=GRB.BINARY, name='players') 
    
    
    
    #add position constraint. We set upper bound to be one and keep the constraint a continuous variable since the 
    #model will keep integer solutions
    
    #positionConstratint = m.addVars(positionConstraints, ub =1 , name='positionConstr')
    
    
    # set objective funcition
    m.setObjective(  - gp.quicksum( returnPlayer[choice] *players[choice,retrn,var] for choice, retrn, var in players ) 
                   + gp.quicksum( playerVariance[choice] *players[choice,retrn,var] for choice, retrn, var in players ) 
                   ,GRB.MINIMIZE)
    
    #constraint: sum of each position must be a constraint:
    #for pos in DF['position'].unique():
        
        
    posConstrs = {} 
    for pos in DF['position'].unique():
        players_in_pos = [playr for playr in player if position[playr] == pos]
        
        posConstrs[pos] = m.addConstr( gp.quicksum( [ players.select(playr)[0] for playr in player if position[playr] == pos ] ) == positionConstraints[pos] )
        
  #  modelPositionConstr =  m.addConstrs(  players.sum(position[choice] ) ==  positionConstraints [ position[choice] ] for choice ,_,_ in players  ) 
    
    #salary constraint
    costConstr = m.addConstr(gp.quicksum( [ playerCost[playr] *players.select(playr)[0] for playr in player ]) <= 50000 )
    
    
    m.update()
    
    m.optimize()
    
    findPlayers = m.X
    
    
    teamRoster = [ player[i] for i in range(len(player)) if findPlayers[i]==1 ]
    print(teamRoster)
    
    sum = 0
    for i in teamRoster:
        sum += playerCost[i] 
    sum 


    constr = { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1} 

    for pos in DF['position'].unique():
        for i in teamRoster:
            if position[i] == pos:
                constr[pos] -= 1
    for key,value in constr.items():
        if value != 0:
            print('failure to meet pos constraint:',key)
    print('valid roster, cost =',sum)    
    
    print('\n expected return - risk :',m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 6 rows, 478 columns and 956 nonzeros
Model fingerprint: 0xf093b841
Variable types: 0 continuous, 478 integer (478 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [4e-14, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 1 rows and 173 columns
Presolve time: 0.00s
Presolved: 5 rows, 305 columns, 601 nonzeros
Variable types: 0 continuous, 305 integer (305 binary)
Found heuristic solution: objective 8.6657800
Found heuristic solution: objective -13.2332400

Root relaxation: objective -1.392026e+01, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  -13.92026    0    2  -13.23324  -13.92026  5.19%     -    0s
H    0     0                     -13.6255733  -13.92026  2.16%     -    0s
     0     0

In [62]:
positionConstraints[position['Arizona']]

1

In [63]:
[players.select(playr)[0] for playr in player if position[playr] == 'DEF']

[<gurobi.Var players[Arizona,2.7,0.7499999999998916] (value 0.0)>,
 <gurobi.Var players[Atlanta,1.3,0.9999999999999832] (value -0.0)>,
 <gurobi.Var players[Baltimore,5.6,0.7999999999998987] (value 0.0)>,
 <gurobi.Var players[Buffalo,12.2,1.8000000000000256] (value 0.0)>,
 <gurobi.Var players[Carolina,13.2,1.0000000000000502] (value 1.0)>,
 <gurobi.Var players[Chicago,5.4,1.8000000000000205] (value 0.0)>,
 <gurobi.Var players[Cincinnati,3.6,1.449999999999958] (value -0.0)>,
 <gurobi.Var players[Cleveland,6.0,1.0000000000000804] (value 0.0)>,
 <gurobi.Var players[Dallas,6.6,0.7999999999998321] (value 0.0)>,
 <gurobi.Var players[Denver,4.6,1.4500000000000524] (value 0.0)>,
 <gurobi.Var players[Detroit,8.4,1.3000000000000878] (value 0.0)>,
 <gurobi.Var players[Green Bay,8.7,1.3999999999999209] (value 0.0)>,
 <gurobi.Var players[Houston,7.9,1.4000000000000123] (value 0.0)>,
 <gurobi.Var players[Indianapolis,7.4,0.7000000000000279] (value -0.0)>,
 <gurobi.Var players[Jacksonville,8.8,2.75000

In [64]:
gp.quicksum( [ playerCost[playr] *players.select(playr)[0] for playr in player ]) <= 50000

<gurobi.TempConstr: <gurobi.LinExpr: 4550.0 players[Arizona,2.7,0.7499999999998916] + 3850.0 players[Atlanta,1.3,0.9999999999999832] + 5000.0 players[Baltimore,5.6,0.7999999999998987] + 5000.0 players[Buffalo,12.2,1.8000000000000256] + 4700.0 players[Carolina,13.2,1.0000000000000502] + 4500.0 players[Chicago,5.4,1.8000000000000205] + 4300.0 players[Cincinnati,3.6,1.449999999999958] + 5000.0 players[Cleveland,6.0,1.0000000000000804] + 4900.0 players[Dallas,6.6,0.7999999999998321] + 4600.0 players[Denver,4.6,1.4500000000000524] + 4800.0 players[Detroit,8.4,1.3000000000000878] + 4700.0 players[Green Bay,8.7,1.3999999999999209] + 4650.0 players[Houston,7.9,1.4000000000000123] + 4500.0 players[Indianapolis,7.4,0.7000000000000279] + 4750.0 players[Jacksonville,8.8,2.7500000000000018] + 4700.0 players[Kansas City,6.2,1.7499999999999478] + 4850.0 players[LA Chargers,6.9,1.5000000000000142] + 5000.0 players[LA Rams,9.4,1.6500000000000346] + 4500.0 players[Miami,2.2,0.800000000000012] + 4650.0 p

In [69]:
teamRoster

sum = 0
for i in teamRoster:
    sum += playerCost[i] 
sum 


constr = { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1} 

for pos in DF['position'].unique():
    for i in teamRoster:
        if position[i] == pos:
            constr[pos] -= 1
for key,value in constr.items():
    if value != 0:
        print('failure to meet pos constraint:',key)
print('valid roster, cost =',sum)


valid roster, cost = 49000.0
